## Random no. generator SQL vs Pandas

In [1]:
query = """Select rowid from fecdata
            order by random() limit 5"""

query = """Select * from fecdata where rowid in (Select rowid from fecdata
            order by random() limit 5)"""

#Same in Pandas 
#df.sample(5)

## Describe vs Sql group by

In [5]:
#fecdata['contb_receipt_amt'].describe()


# pd.read_sql("""
# SELECT COUNT(*), AVG(contb_receipt_amt), MIN(contb_receipt_amt), MAX(contb_receipt_amt)
#     FROM fecdata
# """, fecconn)

## Unique values 

In [6]:
# df['col'].unique() ## this will return array

# pd.read_sql("Select distinct col from table",fecconn) this will return a dataframe if need array

# pd.read_sql("Select distinct col from table",fecconn)['col'].values ## this will return array

## Mapping (defining col. based on already existing col.)

In [7]:
## Pandas

In [9]:
# party_affiliations = {name: 'D' if name == 'Obama, Barack' else 'R' for name in unique_candidates}
# party_affiliations

# candidate_sample = fecdata['cand_nm'].sample(5)
# candidate_sample

# candidate_sample.map(party_affiliations)
# fecdata['party'] = fecdata['cand_nm'].map(party_affiliations)

In [10]:
## Sql
# SELECT
#    ... (column value expressions) ...,
#    CASE column_value_expression
#        WHEN value0 THEN result0
#        WHEN value1 THEN result1
#        ...
#    END AS result_column_name,
#    ... (other column value expressions) ...
#    FROM
#        ...

In [11]:
# when_clauses = ' '.join([f'WHEN "{name}" THEN "{party}"' for name, party in party_affiliations.items()])
# query = f"SELECT cand_nm, CASE cand_nm {when_clauses} END AS party FROM fecdata"
# pd.read_sql(query, fecconn)

## Group by 

In [12]:
#fecdata.groupby('party')['contb_receipt_amt'].sum() # millions of dollars
# SELECT party, sum(contb_receipt_amt)*1e-6 FROM fecdata GROUP BY party

## Value_Counts

In [13]:
# fecdata.groupby('party')['party'].count()
# fecdata['party'].value_counts()

## Mapping incorrect names to correct names in Pandas  Very imp. map with get

In [14]:
# https://colab.research.google.com/drive/104Vxx-76A0MmPqrZ-pBe3j0HjobbJS7a?usp=sharing#scrollTo=dvNFVWzqaGDJ

#len(fecmain['contbr_occupation'].unique()) ## 33006 33k unique values in occupation
# # fecmain['contbr_occupation'].value_counts() - need to club some values

# occ_mapping = {'INFORMATION REQUESTED': 'NOT PROVIDED',
#                'INFORMATION REQUESTED PER BEST EFFORTS': 'NOT PROVIDED',
#                'INFORMATION REQUESTED (BEST EFFORTS)': 'NOT PROVIDED',
#                'C.E.O.': 'CEO'}

#fecmain['contbr_occupation'].map(occ_mapping) ## this will give na for other keys

## to rectify this use get
## fecmain['contbr_occupation'] = fecmain['contbr_occupation'].map(lambda x: occ_mapping.get(x, x))


In [15]:
### Same is sql
## Select col., case when col in ('info..') then X when col in... else col end as modified_col from table

## Pivot table sql (FILTER) vs Pandas

In [16]:
# by_occ = fecmain.pivot_table('contb_receipt_amt', index='contbr_occupation', columns='party',
#                              aggfunc='sum')
# by_occ.sample(5)


## Pivot in sql
# query = """
# SELECT contbr_occupation,
#        SUM(contb_receipt_amt) FILTER (WHERE cand_nm = 'Obama, Barack') AS D,
#        SUM(contb_receipt_amt) FILTER (WHERE cand_nm = 'Romney, Mitt') AS R
# FROM fecdata
# GROUP BY contbr_occupation
# """
# pd.read_sql_query(query, fecconn).sample(3)

## Sorting

In [17]:
# over_1mil.sum(axis=1).sort_values()
# query = """
# SELECT contbr_occupation,
#        SUM(contb_receipt_amt*1e-6) FILTER (WHERE cand_nm = 'Obama, Barack') AS D,
#        SUM(contb_receipt_amt*1e-6) FILTER (WHERE cand_nm = 'Romney, Mitt') AS R
# FROM fecdata
# GROUP BY contbr_occupation
# HAVING D >= 1
# ORDER BY D DESC
# LIMIT 10
# """
# pd.read_sql_query(query, fecconn)

## Ranking

In [21]:
# grouped = fecmain.groupby('cand_nm')
# grouped['contb_receipt_amt'].nlargest(3)

# def top_amounts_by_occupation(df, n=5):
#     totals = df.groupby('contbr_occupation')['contb_receipt_amt'].sum()
#     return totals.nlargest(n)

# top_amounts_by_occupation(fecmain)
#grouped.apply(top_amounts_by_occupation, n=10)

## Binning

In [26]:
#For each of the leading two candidates, did most of their money come from large or small donations?

In [22]:
bins = [0] + [10**k for k in range(0, 8)]
bins

[0, 1, 10, 100, 1000, 10000, 100000, 1000000, 10000000]

In [24]:
# labels = pd.cut(fecmain['contb_receipt_amt'], bins, right=False)
# labels[:5]

In [25]:
# grouped = fecmain.groupby(['cand_nm', labels])
# grouped.size()